In [3]:
# We will:
# 1. Download the FoodSeg103 dataset from Hugging Face.
# 2. Go through each image and its segmentation label (mask).
# 3. Save:
#    - The original image as a JPG.
#    - A *binary* mask as a PNG (0 = background, 1 = any kind of food).
# https://xiongweiwu.github.io/foodseg103.html
from datasets import load_dataset
import os
from PIL import Image
import numpy as np

ds = load_dataset("EduardoPacheco/FoodSeg103")
out_root = "foodseg103_export"

for split in ["train", "validation"]:
    img_dir = f"{out_root}/{split}/images"
    mask_dir = f"{out_root}/{split}/masks"

    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)

    for i, row in enumerate(ds[split]):
        img = row["image"]
        mask = row["label"]

        img_path = os.path.join(img_dir, f"{i:05d}.jpg")
        img.save(img_path)

        mask = np.array(mask)

        bin_mask = np.zeros_like(mask, dtype=np.uint8)
        bin_mask[mask > 0] = 1
        bin_mask_img = Image.fromarray(bin_mask)

        mask_path = os.path.join(mask_dir, f"{i:05d}.png")
        bin_mask_img.save(mask_path)

print("Done! Masks now use 0=background, 1=food.")


Done! Masks now use 0=background, 1=food.


In [ ]:
# We want to:
#   - Keep ONLY the samples where there is exactly ONE card in the mask (one card class present).
# https://www.kaggle.com/datasets/luanademi/playing-card-ensemble-segmentation-masks?utm_source=chatgpt.com
import os
import pickle
import numpy as np
from PIL import Image

PCK_PATH = r"C:\Users\siuts\Downloads\asm\CS489\project\scenes.pck"   # change if needed
OUT_ROOT = "single_card_52"
IMG_DIR  = os.path.join(OUT_ROOT, "images")
MASK_DIR = os.path.join(OUT_ROOT, "masks")

os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(MASK_DIR, exist_ok=True)

# 3. Load pickle
with open(PCK_PATH, "rb") as f:
    obj = pickle.load(f)

images = obj["data"]
masks  = obj["gt"]

n_img  = len(images)
n_mask = len(masks)

def get_item(x, i):
    return x[i]

saved_count = 0

for i in range(n_img):
    img = np.array(images[i])
    m   = np.array(masks[i])

    uniq = np.unique(m)
    non_zero_classes = uniq[uniq > 0]

    # we only want masks with EXACTLY ONE non-zero class (one card)
    if len(non_zero_classes) != 1:
        continue
    
    mask_to_save = m.astype(np.uint8)
    fname = f"{i:06d}.png"
    Image.fromarray(img).save(os.path.join(IMG_DIR,  fname))
    Image.fromarray(mask_to_save).save(os.path.join(MASK_DIR, fname))

    saved_count += 1

print(f"Done! Saved {saved_count} single-card samples into '{OUT_ROOT}/images' and '{OUT_ROOT}/masks'.")


Done! Saved 831 single-card samples into 'single_card_52/images' and 'single_card_52/masks'.


In [ ]:
# we want to:
#   1. Turn this into a *binary* problem:
#         0 = background
#         2 = "card" (any card class)
#      (So all card labels 1..52 become just 2.)
#   2. Split the data into:
#         - 60% training set
#         - 40% validation set
#   3. Save everything into this folder structure:
#         single_card_2/
#             train/
#                 images/
#                 masks/
#             validation/
#                 images/
#                 masks/
import os
import numpy as np
from PIL import Image

# Source (multi-class, single-card) dataset
SRC_ROOT = "single_card_52"
SRC_IMG_DIR  = os.path.join(SRC_ROOT, "images")
SRC_MASK_DIR = os.path.join(SRC_ROOT, "masks")

# Target (binary card=2) dataset with train/val
OUT_ROOT = "single_card_2"
TRAIN_IMG_DIR = os.path.join(OUT_ROOT, "train", "images")
TRAIN_MASK_DIR = os.path.join(OUT_ROOT, "train", "masks")
VAL_IMG_DIR   = os.path.join(OUT_ROOT, "validation", "images")
VAL_MASK_DIR  = os.path.join(OUT_ROOT, "validation", "masks")

os.makedirs(TRAIN_IMG_DIR, exist_ok=True)
os.makedirs(TRAIN_MASK_DIR, exist_ok=True)
os.makedirs(VAL_IMG_DIR, exist_ok=True)
os.makedirs(VAL_MASK_DIR, exist_ok=True)

mask_files = sorted([f for f in os.listdir(SRC_MASK_DIR) if f.lower().endswith(".png")])
n = len(mask_files)

# 60% train, 40% validation
split_idx = int(n * 0.6)

converted = 0

for k, fname in enumerate(mask_files):
    mask_path = os.path.join(SRC_MASK_DIR, fname)
    img_path  = os.path.join(SRC_IMG_DIR,  fname)

    img = Image.open(img_path).convert("RGB")
    img_np = np.array(img)

    m = Image.open(mask_path)
    m_np = np.array(m)

    m_np = m_np[..., 0]
    m_bin = np.zeros_like(m_np, dtype=np.uint8)
    m_bin[m_np > 0] = 2 

    if k < split_idx:
        out_img_path  = os.path.join(TRAIN_IMG_DIR,  fname)
        out_mask_path = os.path.join(TRAIN_MASK_DIR, fname)
    else:
        out_img_path  = os.path.join(VAL_IMG_DIR,  fname)
        out_mask_path = os.path.join(VAL_MASK_DIR, fname)

    Image.fromarray(img_np).save(out_img_path)
    Image.fromarray(m_bin).save(out_mask_path)

    converted += 1

print(f"Done! Converted {converted} samples into '{OUT_ROOT}/train' and '{OUT_ROOT}/validation'.")


Done! Converted 831 samples into 'single_card_2/train' and 'single_card_2/validation'.


In [ ]:
# We want to COMBINE (merge) these two datasets into one big dataset.
#
# The final structure will look like:
#   merged_dataset/
#       train/
#           images/
#           masks/
#       validation/
#           images/
#           masks/
import os
import shutil
from PIL import Image
import numpy as np

# Paths
FOOD_ROOT = "foodseg103_export"
CARD_ROOT = "single_card_2"
OUT_ROOT  = "merged_dataset"

# Output folders
TRAIN_IMG = os.path.join(OUT_ROOT, "train", "images")
TRAIN_MASK = os.path.join(OUT_ROOT, "train", "masks")
VAL_IMG = os.path.join(OUT_ROOT, "validation", "images")
VAL_MASK = os.path.join(OUT_ROOT, "validation", "masks")

os.makedirs(TRAIN_IMG, exist_ok=True)
os.makedirs(TRAIN_MASK, exist_ok=True)
os.makedirs(VAL_IMG, exist_ok=True)
os.makedirs(VAL_MASK, exist_ok=True)

# helper
def list_sorted(folder):
    return sorted(os.listdir(folder))

def copy_pair(src_img, src_mask, dst_img, dst_mask):
    shutil.copy2(src_img, dst_img)
    shutil.copy2(src_mask, dst_mask)

# -----------------------------------------------------
# MERGE TRAIN SPLITS
# -----------------------------------------------------
counter = 0
print("Merging TRAIN...")

# 1. FOOD train
food_train_imgs = list_sorted(os.path.join(FOOD_ROOT, "train", "images"))
for name in food_train_imgs:
    img_path = os.path.join(FOOD_ROOT, "train", "images", name)
    mask_path = os.path.join(FOOD_ROOT, "train", "masks", name.replace(".jpg", ".png"))

    out_img  = os.path.join(TRAIN_IMG,  f"{counter:06d}.jpg")
    out_mask = os.path.join(TRAIN_MASK, f"{counter:06d}.png")

    copy_pair(img_path, mask_path, out_img, out_mask)
    counter += 1

# 2. CARD train
card_train_imgs = list_sorted(os.path.join(CARD_ROOT, "train", "images"))
for name in card_train_imgs:
    img_path = os.path.join(CARD_ROOT, "train", "images", name)
    mask_path = os.path.join(CARD_ROOT, "train", "masks", name)

    out_img  = os.path.join(TRAIN_IMG,  f"{counter:06d}.png")
    out_mask = os.path.join(TRAIN_MASK, f"{counter:06d}.png")

    copy_pair(img_path, mask_path, out_img, out_mask)
    counter += 1

print(f"TRAIN merged: {counter} items")

# -----------------------------------------------------
# MERGE VALIDATION SPLITS
# -----------------------------------------------------
counter = 0
print("Merging VALIDATION...")

# 1. FOOD val
food_val_imgs = list_sorted(os.path.join(FOOD_ROOT, "validation", "images"))
for name in food_val_imgs:
    img_path = os.path.join(FOOD_ROOT, "validation", "images", name)
    mask_path = os.path.join(FOOD_ROOT, "validation", "masks", name.replace(".jpg", ".png"))

    out_img  = os.path.join(VAL_IMG,  f"{counter:06d}.jpg")
    out_mask = os.path.join(VAL_MASK, f"{counter:06d}.png")

    copy_pair(img_path, mask_path, out_img, out_mask)
    counter += 1

# 2. CARD val
card_val_imgs = list_sorted(os.path.join(CARD_ROOT, "validation", "images"))
for name in card_val_imgs:
    img_path = os.path.join(CARD_ROOT, "validation", "images", name)
    mask_path = os.path.join(CARD_ROOT, "validation", "masks", name)

    out_img  = os.path.join(VAL_IMG,  f"{counter:06d}.png")
    out_mask = os.path.join(VAL_MASK, f"{counter:06d}.png")

    copy_pair(img_path, mask_path, out_img, out_mask)
    counter += 1

print(f"VALIDATION merged: {counter} items")

print("Done! Merged dataset is in merged_dataset/")


In [ ]:
# we want to create YOLO-style LABEL FILES for segmentation (polygons).
# where:
#   - class_id is in YOLO format:
#         0 = food
#         1 = card
#
# Final structure:
#   merged_dataset/
#       train/
#           images/
#           masks/
#           labels/
#       validation/
#           images/
#           masks/
#           labels/
import os
import numpy as np
import cv2
from tqdm import tqdm

# New dataset root
ROOT = r"C:\Users\siuts\Downloads\asm\CS489\project\merged_dataset"
splits = ["train", "validation"]

def mask_to_polygons(mask):
    """Convert mask (H,W) into class→polygons dict."""
    polygons = {}
    h, w = mask.shape

    class_ids = np.unique(mask)
    class_ids = class_ids[class_ids != 0]  # remove background (0)

    for cid in class_ids:
        binary = (mask == cid).astype(np.uint8) * 255

        contours, _ = cv2.findContours(
            binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )

        polygon_list = []
        for c in contours:
            poly = c.reshape(-1, 2)
            px = poly[:, 0] / w
            py = poly[:, 1] / h

            poly_norm = np.vstack((px, py)).T.reshape(-1).tolist()
            polygon_list.append(poly_norm)

        if polygon_list:
            polygons[cid] = polygon_list

    return polygons


for split in splits:
    mask_dir = os.path.join(ROOT, split, "masks")
    label_dir = os.path.join(ROOT, split, "labels")
    os.makedirs(label_dir, exist_ok=True)

    mask_files = sorted(os.listdir(mask_dir))
    print(f"Processing {split} split...")

    for fname in tqdm(mask_files):
        mask_path = os.path.join(mask_dir, fname)
        img_id = os.path.splitext(fname)[0]
        label_path = os.path.join(label_dir, f"{img_id}.txt")

        # load mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        polygons = mask_to_polygons(mask)

        with open(label_path, "w") as f:
            for cid, polys in polygons.items():
                # MAP mask class → YOLO class
                # mask: 1 = food, 2 = card
                # YOLO: 0 = food, 1 = card
                if cid == 1:
                    yolo_cid = 0
                elif cid == 2:
                    yolo_cid = 1

                for poly in polys:
                    line = str(yolo_cid) + " " + " ".join(f"{p:.6f}" for p in poly)
                    f.write(line + "\n")


Processing train split...


100%|██████████| 5481/5481 [00:40<00:00, 134.01it/s]


Processing validation split...


100%|██████████| 2468/2468 [00:06<00:00, 360.95it/s]
